In [2]:
import cv2
import matplotlib.pyplot as plt
from threading import *
import json
import tkinter
from tkinter import *
import random
import time
from win10toast import ToastNotifier
toaster=ToastNotifier()
j=0

In [3]:
config_file= 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model= 'frozen_inference_graph.pb'

In [4]:
model = cv2.dnn_DetectionModel(frozen_model,config_file)

In [5]:
classLabels= []
file_name ='Label.txt'
with open(file_name,'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')

In [6]:
model.setInputSize(120,120)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)

<dnn_Model 000000D4ED69C0F0>

# Video

In [9]:
arr=[]

class Proct(Thread):
    def run(self):
          cap=cv2.VideoCapture(1)

          if not cap.isOpened():
            cap = cv2.VideoCapture(0)
          if not cap.isOpened():
            raise IOError('cannot open webcam')
        
          font_scale=2
          font= cv2.FONT_HERSHEY_COMPLEX_SMALL
          
          while True:
            ret,frame=cap.read()
        
            ClassIndex, confidence, bbox= model.detect(frame,confThreshold=0.55)
        
            print(ClassIndex)
            if (len(ClassIndex)!=0):
                for ClassInd, conf, boxes in zip(ClassIndex.flatten(),confidence.flatten(),bbox):
                    if(ClassInd<=77 and ClassInd>0):
                        cv2.rectangle(frame,boxes,(0,255,0),2)
                        cv2.putText(frame,classLabels[ClassInd-1],(boxes[0]+6,boxes[1]+30),font,fontScale=font_scale,color=(255,0,0), thickness=1)
                        arr.append(classLabels[ClassInd-1])
                        #labelresulttext.destroy()
                        if(classLabels[ClassInd-1]=='Mobile Phone' or classLabels[ClassInd-1]=='Laptop'):
                            """
                            j=1;
                            labelresulttext = Label(
                            root,
                            font = ("Consolas",20),
                            background = "#8B2323",
                            )
                            labelresulttext.pack()
                            labelresulttext.configure(text="Warning: Electronic gadget was detected")*/
                            """
                            
                            toaster.show_toast("Warning","Either electronic gadget or other anonymous activity detected",threaded=True,icon_path=None,duration=5)
                            while(toaster.notification_active()):
                                 time.sleep(0.1)
                            
                            
            cv2.imshow('Detection',frame)
        
            if cv2.waitKey(2) & 0xFF== ord('q'):
                break;
      
          cap.release()
          cv2.destroyAllWindows()

In [ ]:
t1=Proct()
with open('./data.json', encoding="utf8") as f:
    data = json.load(f)

# convert the dictionary in lists of questions and answers_choice 
questions = [v for v in data[0].values()]
answers_choice = [v for v in data[1].values()]

answers = [1,1,1,1,3,1,0,1,3,3] 

user_answer = []

indexes = []
def gen():
    global indexes
    while(len(indexes) < 5):
        x = random.randint(0,9)
        if x in indexes:
            continue
        else:
            indexes.append(x)


def showresult(score):
    lblQuestion.destroy()
    r1.destroy()
    r2.destroy()
    r3.destroy()
    r4.destroy()
    labelimage = Label(
        root,
        background = "#ffffff",
        border = 0,
    )
    labelimage.pack(pady=(10,40))
    labelresulttext = Label(
        root,
        font = ("Consolas",20),
        background = "#FFFFFF",
        foreground="#FF0000"
    )
    labelresulttext.pack()
    
    for i in arr:
        if(i=='Mobile Phone'):
            labelresulttext.configure(text="You have got multiple Warnings\n\n")
            break;
    """
    if score >= 20:
          labelresulttext.configure(text="Your score is ")
    elif (score >= 10 and score < 20):
            labelresulttext.configure(text="Your score is saved")
    else:
            labelresulttext.configure(text="Your score is saved")
    #labelresulttext.configure(text="Total warnings: ")
    """
    labelres = Label(
        root,
        font = ("Consolas",25),
        background = "#FFFFFf",
    )
    labelres.pack(pady=(0,250))
    labelres.configure(text="Your score is saved We will get back to you \n\n\n\n\nThank you")
    
    

def calc():
    global indexes,user_answer,answers
    x = 0
    score = 0
    i=0
    k=0
   # for i in arr:
   #     if(i=='Mobile Phone'):
    #       k=1
   # if(k==1):
  #  print("Mobile Phone was Detected")
    for i in indexes:
        if user_answer[x] == answers[i]:
            score = score + 5
        x += 1
    print(score)
    showresult(score)


ques = 1
def selected():
    global radiovar,user_answer
    global lblQuestion,r1,r2,r3,r4
    global ques
    x = radiovar.get()
    user_answer.append(x)
    radiovar.set(-1)
    if ques < 5:
        lblQuestion.config(text= questions[indexes[ques]])
        r1['text'] = answers_choice[indexes[ques]][0]
        r2['text'] = answers_choice[indexes[ques]][1]
        r3['text'] = answers_choice[indexes[ques]][2]
        r4['text'] = answers_choice[indexes[ques]][3]
        ques += 1
        calc()
    else:
        # print(indexes)
        # print(user_answer)
        # these two lines were just developement code
        # we don't need them
        calc()
    

def startquiz():
    global lblQuestion,r1,r2,r3,r4
    lblQuestion = Label(
        root,
        text = questions[indexes[0]],
        font = ("Consolas", 16),
        width = 500,
        justify = "center",
        wraplength = 400,
        background = "#ffffff",
    )
    lblQuestion.pack(pady=(100,30))

    global radiovar
    radiovar = IntVar()
    radiovar.set(-1)

    r1 = Radiobutton(
        root,
        text = answers_choice[indexes[0]][0],
        font = ("Times", 12),
        value = 0,
        variable = radiovar,
        command = selected,
        background = "#ffffff",
    )
    r1.pack(pady=5)

    r2 = Radiobutton(
        root,
        text = answers_choice[indexes[0]][1],
        font = ("Times", 12),
        value = 1,
        variable = radiovar,
        command = selected,
        background = "#ffffff",
    )
    r2.pack(pady=5)

    r3 = Radiobutton(
        root,
        text = answers_choice[indexes[0]][2],
        font = ("Times", 12),
        value = 2,
        variable = radiovar,
        command = selected,
        background = "#ffffff",
    )
    r3.pack(pady=5)

    r4 = Radiobutton(
        root,
        text = answers_choice[indexes[0]][3],
        font = ("Times", 12),
        value = 3,
        variable = radiovar,
        command = selected,
        background = "#ffffff",
    )
    r4.pack(pady=5)


def startIspressed():
    labelimage.destroy()
    labeltext.destroy()
    lblInstruction.destroy()
    lblRules.destroy()
    btnStart.destroy()
    gen()
    startquiz()
    t1.start()



root = tkinter.Tk()
root.title("Online Exam")
root.geometry("1350x725")
root.config(background="#ffffff")
root.resizable(0,0)



labelimage = Label(
    root,
    background = "#ffffff",
)
labelimage.pack(pady=(40,0))

labeltext = Label(
    root,
    text = "Online Exam",
    font = ("Comic sans MS",24,"bold"),
    background = "#ffffff",
)
labeltext.pack(pady=(0,50))

img2 = PhotoImage(file="Frame.png")

btnStart = Button(
    root,
    image = img2,
    relief = FLAT,
    border = 0,
    command = startIspressed,
)
btnStart.pack()

lblInstruction = Label(
    root,
    text = "Click Start Once You Are ready",
    background = "#ffffff",
    font = ("Consolas",14),
    justify = "center",
)
lblInstruction.pack(pady=(10,100))

lblRules = Label(
    root,
    text = "Remember you are being monitored ",
    width = 100,
    font = ("Times",14),
    background = "#000000",
    foreground = "#FACA2F",
)
lblRules.pack()

root.mainloop()



()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
[[1]]
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
[[1]]
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
[[10]]
()
()
[[77]]


In [14]:
arr.clear()
